# Creating Type Classes and Instances

## Outline

* Revisiting Type Classes
* Exploring pre-existent type Classes
	* The `Eq` type class
	* The `Num` type class
* Deriving
* Type Classes
	* Classees
	* Instances
	* Subclass
	* Complete example ?


In this lesson, first, we'll do a quick recap of type classes, learn how to create them from scratch, and how to use them.

## Revisiting Type Classes

* Type classes are interfaces that define some behavior. (For example, comparing for equality.)

* If a type is an instance of a type class, it supports the behaviors of that type class. Meaning that we can use the functions the type class defines.

* A type class can have multiple instances, and a type can belong to multiple type classes.

## Exploring pre-existent type classes

### Defining the `Eq` type class

Remember that the `Eq` type class is all about equality. The types that are instances of the `Eq` type class can say if two values of its type are equal or different by using the `==` (equal) and `/=` (not equal) functions.

But how do we define the `Eq` class? We can do it ike this:

```haskell
class Eq a where
  (==) :: a -> a -> Bool
  (/=) :: a -> a -> Bool
```

* `class ... where` means that we're defining a new type class.
* `Eq` is how type class will be called.
* The `a` inside `Eq a` is the type variable that holds the place to the type that will be made an instance of this type class.
* Then, we define the functions of the type class (`==` and `/=`).

But wait.. the functions aren't implemented!

That's because, you can do it, but there's no **need** to actually implement the functions in the type class. **Only the type delcarations are required!**

Now that we have a type class, let's use it! 

### Defining an instance for the `Eq` type class

Let's define a type for the payment methods a customer can use:

In [ ]:
data PaymentMethod = Cash | Card | CC -- CC stands for Cryptocurrency

Now, lets make `PaymentMethod` an instance of `Eq`, so we can check its values for equality.

In [ ]:
instance Eq PaymentMethod where
  -- Implementation of ==
  Cash == Cash = True
  Card == Card = True
  CC == CC = True
  _ == _ = False
  
  -- Implementation of /=
  Cash /= Cash = False
  Card /= Card = False
  CC /= CC = False
  _ /= _ = True

* `instance ... where` means that we're making some type an instance of a type class
* `Eq PaymentMethod` means that `PaymentMethod` is the type that will be come an instance of `Eq` type class. 
* Then, we implement the functions of the type class (`==` and `/=`) for our specific type.

As you can see, because now we're creating an instance for a type, we replace the type variable (`a`) that we had in the type class definition for that specific type (`PaymentMethod`).

And that's it! That's how you define a type class and how you make a type an instance of it! Now, `PaymentMethod` can freely use the `Eq` behaviours (`==` and `/=`):

In [ ]:
Card == Cash
CC /= Card

But here's the thing. We're on our way to become epic Haskell developers, and epic Haskell developers can do better than that.

### Improving our `Eq` type class

With our `Eq` type class defined like that, each time we need to make a type an instance of `Eq`, we have to define both `==` and `/=`. But we know that we can also define functions inside the type class definition, and we know that for each pair of values, `==` and `/=` give use the opposite `Bool` value. 

So, taking into account the relationship between `==` and `/=`, we can redefine `Eq` type class like this:

```haskell
class  Eq a  where
    (==), (/=)  :: a -> a -> Bool
    x /= y      = not (x == y)
    x == y      = not (x /= y)
```

Which is how `Eq` is actually defined in the standard library!

By defining `==` and `/=` as the oposite of the other one (mutual recursion), now Haskell can infer the behavior of one from the other. And each time we make a type an instance of the `Eq` type class, we just need to overwrite (implement) one of them! Like this:

In [ ]:
instance Eq PaymentMethod where
  Cash == Cash = True
  Card == Card = True
  CC == CC = True
  _ == _ = False

And the type can use both `==` and `/=` no questions asked!

This is called **minimal complete definition**. 

You can take advantage of this by checking what's the minimal complete definition of a type class using `:i <type class>` and implementing only those behaviors. If you do `:i Eq`, you'll get:

```haskell
type Eq :: * -> Constraint -- Eq takes a concrete type and  returns a Constraint
class Eq a where
  (==) :: a -> a -> Bool
  (/=) :: a -> a -> Bool
  {-# MINIMAL (==) | (/=) #-}
  	-- Defined in ‘GHC.Classes’

-- ... and a bunch of instances.
```

In this line:

```haskell
{-# MINIMAL (==) | (/=) #-}
```

It says that to have the *minimal complete implementation* of the class, you have to implement either `==` OR `/=`.

In [ ]:
-- Check by yourself!
:i Eq

## Deriving

Deriving can go wrong (see: https://www.youtube.com/watch?v=1txgSlcpQmo)

---

And that makes sense! Try to come up with a single implementation that works with every type.

A single implementation of `==` that works for `Int`, `Char`, lists, etc.

Different types need different implementations

Where do we implement the functions then? Well, we can implement them them at the type class itself, at the instances, or a combination of both. As an example, I'll show you how the real way of how the `Eq` type class is defined in Haskell (Yes, I just lied to you):

How `Eq` is actually defined in the standard library (minus details):

```haskell
class  Eq a  where
    (==), (/=)           :: a -> a -> Bool
    x /= y               = not (x == y)
    x == y               = not (x /= y)
```

As you can see, we can define multiple functions with the same type declaration in a single line. But more importantly, we just implemented both `==` and `/=`!